# Outils informatiques pour le "big data"

## Contrôle du 14 décembre 2017

### Comptage de mots dans les traductions anglaises de 4 romans francais.

Nous allons travailler avec les 4 textes des auteurs suivants

- Victor Hugo http://www.gutenberg.org/files/135/135-0.txt
- Marcel Proust http://www.gutenberg.org/files/7178/7178-8.txt
- Emile Zola http://www.gutenberg.org/files/1069/1069-0.txt
- Stendhal http://www.gutenberg.org/files/44747/44747-0.txt

Pour ceux qui utilisent le proxy, téléchargez ces fichiers directement depuis votre navigateur. Sinon vous pouvez exécutez la cellule suivante.

In [ ]:
%mkdir -p books
import urllib.request as url
url.urlretrieve("http://www.gutenberg.org/files/135/135-0.txt",     filename="books/hugo.txt")
url.urlretrieve("http://www.gutenberg.org/files/7178/7178-8.txt",   filename="books/proust.txt")
url.urlretrieve("http://www.gutenberg.org/files/1069/1069-0.txt",   filename="books/zola.txt")
url.urlretrieve("http://www.gutenberg.org/files/44747/44747-0.txt", filename="books/stendhal.txt")

### Liste de fichiers

- Créer une liste `filenames` contenant les noms des fichiers contenant les text des 4 livres.

In [ ]:
import os, glob
here = os.getcwd()
filenames = sorted(glob.glob(os.path.join(here, 'books', '*.txt')))                                    

### Fonction `wordcount_map`

- Coder une fonction nommée `wordcount_map` permettant de lire un fichier et de compter 
les mots transformés en minuscules et ne contenant que des lettres de l'alphabet.
- Cette fonction renvoie un dictionnaire dont chaque mot du texte une clé et son
nombre d'occurences la valeur corespondante.

In [ ]:
def wordcount_map(textfile):
                                     
    with open(textfile,'r',encoding='latin-1') as f:
       words = []
       for line in f:
          word_list = line.strip('\n').strip('\r').strip('.').strip(',').split(' ')
          words.append([word.lower() for word in word_list if word.isalpha()])
    
    result = {}

    for sentence in words:
       for word in sentence:
          try:
             result[word] += 1 
          except KeyError:
             result[word] = 1
                
    return result

- Avec une boucle for, appliquer cette fonction sur la liste `filenames`.
- Afficher le nombre de mots différents pour chaque livre

In [ ]:
filenames

### Fonction `wordcount_reduce`

- Coder une fonction avec comme argument une liste de dictionnaires calculés chacun par la fonction `wordcount_map` qui retourne un dictionnaire les mots apparaissant dans **tous** les livres avec leur nombre d'occurences total. 

- Ecrire le code et donner le nombre d'apparitions du mot `valjean` dans `les misérables`.

In [ ]:
import operator

def wordcount_reduce(dict_list):
    result = {}
    for d in dict_list:
        for k, v in d.items():
            try:
                result[k] += v
            except KeyError:
                result[k] = v

    return result

In [ ]:
%%time

import operator
dict_list = [wordcount_map(filenames[0])] 
counts = wordcount_reduce(dict_list)
result = sorted(counts.items(), key=operator.itemgetter(1), reverse=True)

In [ ]:
for mot, occurence in result:
    if mot == 'valjean':
        print(occurence)

### Version sequentielle

- Ecrire la boucle `for` qui permet de calculer les occurences des mots dans les 4 livres.
- Afficher les 5 premiers mots les plus courants.

In [ ]:
import itertools
mapped_values = []
for f in filenames:
   mapped_values.append(wordcount_map(f))

result = wordcount_reduce(mapped_values)
sorted(result.items(),key=operator.itemgetter(1), reverse=True)[:5]

- Remplacer cette boucle en utilisant `map`

In [ ]:
import itertools
mapped_values = map(wordcount_map,filenames)

result = wordcount_reduce(mapped_values)
sorted(result.items(),key=operator.itemgetter(1), reverse=True)[:5]

### Version parallèle

- Paralléliser l'étape utilisant `wordcount_map` sur 4 processus avec concurrent.futures

In [ ]:
%%time

from concurrent.futures import ProcessPoolExecutor
e = ProcessPoolExecutor(4)

mapped_values = list(e.map(wordcount_map,filenames))
result = wordcount_reduce(mapped_values)

In [ ]:
sorted(result.items(),key=operator.itemgetter(1), reverse=True)[:5]

### PySpark

- Paralleliser la boucle avec PySpark

In [ ]:
import pyspark

sc = pyspark.SparkContext('local[4]')

In [ ]:
%%time

rdd = sc.parallelize(filenames)
mapped_values = rdd.map(wordcount_map).collect()

In [ ]:
result = wordcount_reduce(mapped_values)

In [ ]:
sorted(result.items(),key=operator.itemgetter(1), reverse=True)[:5]

In [ ]:
sc.stop()

### Dask.bag

- Paralléliser la boucle avec Dask.bag

In [ ]:
import dask.bag as db

bag = db.from_sequence(filenames)
mapped_values = bag.map(wordcount_map).compute()


In [ ]:
result = wordcount_reduce(mapped_values)
sorted(result.items(),key=operator.itemgetter(1), reverse=True)[:5]

### Dask.delayed

- Paralléliser la boucle avec Dask.delayed

In [ ]:
%%time

from dask import delayed, compute
import dask.multiprocessing

delayed_values = [delayed(wordcount_map)(f) for f in filenames]

mapped_values = compute(*delayed_values, get=dask.multiprocessing.get)

In [ ]:
result = wordcount_reduce(mapped_values)
sorted(result.items(),key=operator.itemgetter(1), reverse=True)[:5]

## Bonus: 
 - Donner la liste des mots communs au 4 livres commencant par `q`.

In [ ]:
for w in list(set.intersection(*(set(d.keys()) for d in mapped_values))):
    if w.startswith('q'):
        print(w)